# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,260.510665,0.953046,1234,261.463711,259.557619,522.927422,519.115239
6,239.878404,0.305787,1234,240.184191,239.572617,480.368381,479.145235
7,172.313478,0.137867,1234,172.451345,172.175612,344.902690,344.351223
8,920.080336,-0.002939,1234,920.077397,920.083275,1840.154793,1840.166550
9,35.250241,-0.322033,1234,34.928208,35.572274,69.856416,71.144549
10,389.195127,-0.558725,1234,388.636402,389.753852,777.272803,779.507704
11,523.789748,0.873443,1234,524.663191,522.916305,1049.326381,1045.832611
12,839.268491,0.382206,1234,839.650697,838.886285,1679.301395,1677.772569
13,162.739555,0.919743,1234,163.659298,161.819812,327.318597,323.639624
14,209.961729,-0.105351,1234,209.856378,210.067080,419.712756,420.134160


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-261.463711,259.557619
6,-240.184191,239.572617
7,-172.451345,172.175612
8,-920.077397,920.083275
9,-34.928208,35.572274
10,-388.636402,389.753852
11,-524.663191,522.916305
12,-839.650697,838.886285
13,-163.659298,161.819812
14,-209.856378,210.067080


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-261.463711,259.557619
6,-240.184191,239.572617
7,-172.451345,172.175612
8,-920.077397,920.083275
9,-34.928208,35.572274
10,-388.636402,389.753852
11,-524.663191,522.916305
12,-839.650697,838.886285
13,-163.659298,161.819812
14,-209.856378,210.067080


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-261.46371113,  259.55761949],
       [-240.18419064,  239.57261736],
       [-172.45134497,  172.17561158],
       [-920.07739668,  920.08327514],
       [ -34.92820816,   35.57227428],
       [-388.63640154,  389.75385221],
       [-524.66319064,  522.91630544],
       [-839.65069725,  838.88628457],
       [-163.65929827,  161.81981221],
       [-209.85637784,  210.0670801 ]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,520.927422,-519.115239
6,478.368381,-479.145235
7,342.902690,-344.351223
8,1838.154793,-1840.166550
9,67.856416,-71.144549
10,775.272803,-779.507704
11,1047.326381,-1045.832611
12,1677.301395,-1677.772569
13,325.318597,-323.639624
14,417.712756,-420.134160


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


In [16]:
# normalize inputs to unit domain [0,1]
normed_data = vocs.normalize_inputs(data)
normed_data

,a,b
5,0.260511,0.976523
6,0.239878,0.652893
7,0.172313,0.568933
8,0.920080,0.498530
9,0.035250,0.338983
10,0.389195,0.220637
11,0.523790,0.936721
12,0.839268,0.691103
13,0.162740,0.959872
14,0.209962,0.447324


In [17]:
# and denormalize
vocs.denormalize_inputs(normed_data)

,a,b
5,260.510665,0.953046
6,239.878404,0.305787
7,172.313478,0.137867
8,920.080336,-0.002939
9,35.250241,-0.322033
10,389.195127,-0.558725
11,523.789748,0.873443
12,839.268491,0.382206
13,162.739555,0.919743
14,209.961729,-0.105351


# Error handling

In [18]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [19]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [20]:
data['a']  = np.nan

In [21]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [22]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [23]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
